<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/Gaussian_Splat_w_DINO%2BALIKED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR



# **Fountain: Gaussian Splat w/ DINO+ALIKED**

In [2]:
import subprocess
import sys
import os

INSTALL_DIR = "/content/packages"
os.makedirs(INSTALL_DIR, exist_ok=True)

print("="*60)
print("Building LightGlue only...")
print("="*60)

# Install LightGlue only
# Dependencies (including pycolmap) will not be installed
print("\nInstalling LightGlue (no dependencies)...")
subprocess.run([
    sys.executable, '-m', 'pip', 'install',
    '--target', INSTALL_DIR,
    '--no-deps',  # No dependencies
    'git+https://github.com/cvg/LightGlue.git'
], check=True)

print("\n" + "="*60)
print("✓ Build Complete!")
print("="*60)
print(f"\nPackages built to: {INSTALL_DIR}")
print("\nNote: pycolmap will be installed on-the-fly in the main notebook.")

Building LightGlue only...

Installing LightGlue (no dependencies)...

✓ Build Complete!

Packages built to: /content/packages

Note: pycolmap will be installed on-the-fly in the main notebook.


In [3]:

#サイズの異なる画像を扱う
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import cv2

# Configuration
# IMAGE_PATH: Path to the image folder
# WORK_DIR: Working directory for Gaussian Splatting repository
# OUTPUT_DIR: Directory for the final video output
# COLMAP_DIR: Directory for COLMAP data

IMAGE_DIR = "/content/drive/MyDrive/your_folder/grand_place2"
WORK_DIR = '/content/gaussian_splatting'
OUTPUT_DIR = '/content/output'
COLMAP_DIR = '/content/colmap_data'



In [5]:
!pip install kornia
!pip install pycolmap
!pip install lightglue
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 55.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement lightglue (from versions: none)
ERROR: No matching distribution found for lightglue


In [6]:
# =========================================================
# Cell 1: Setup (Revised Version)
# =========================================================
import sys
import os
import subprocess

# Environment settings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("Setting up environment...")
print("="*60)

# =========================================================
# Kaggle defaults (Packages with C-binaries)
# =========================================================
print("\n[1/5] Loading Kaggle default packages...")
import gc
import glob
from pathlib import Path
from tqdm import tqdm

import numpy as np
import h5py
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

print(f"✓ Defaults loaded: numpy {np.__version__}, torch {torch.__version__}")
print(f"  CUDA Available: {torch.cuda.is_available()}")

# =========================================================
# pycolmap (With C-binaries - Install on-the-fly)
# =========================================================
print("\n[2/5] Installing pycolmap...")
try:
    import pycolmap
    print("✓ pycolmap already available")
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'pycolmap'],
                  check=True)
    import pycolmap
    print("✓ pycolmap installed")

# =========================================================
# LightGlue (Pure Python - Using pre-built package)
# =========================================================
print("\n[3/5] Loading LightGlue...")
PACKAGES_PATH = '/content/lightglue-package-builder/packages'

if os.path.exists(PACKAGES_PATH):
    sys.path.insert(0, PACKAGES_PATH)
    print(f"✓ Using pre-built path: {PACKAGES_PATH}")
else:
    print(f"⚠️  Pre-built not found, installing via pip...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'git+https://github.com/cvg/LightGlue.git'], check=True)

from lightglue import ALIKED, LightGlue
print("✓ LightGlue loaded")

# =========================================================
# transformers
# =========================================================
print("\n[4/5] Loading transformers...")
try:
    from transformers import AutoImageProcessor, AutoModel
    print("✓ transformers loaded")
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'transformers'],
                  check=True)
    from transformers import AutoImageProcessor, AutoModel
    print("✓ transformers installed")

# =========================================================
# COLMAP (System Package)
# =========================================================
print("\n[5/5] Installing COLMAP system binary...")
subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
subprocess.run(['apt-get', 'install', '-y', '-qq', 'colmap'], capture_output=True)
print("✓ COLMAP binary installed")

print("\n" + "="*60)
print("✓ All packages ready!")
print("="*60)

Setting up environment...

[1/5] Loading Kaggle default packages...
✓ Defaults loaded: numpy 2.0.2, torch 2.9.0+cu126
  CUDA Available: True

[2/5] Installing pycolmap...
✓ pycolmap already available

[3/5] Loading LightGlue...
⚠️  Pre-built not found, installing via pip...
✓ LightGlue loaded

[4/5] Loading transformers...
✓ transformers loaded

[5/5] Installing COLMAP system binary...
✓ COLMAP binary installed

✓ All packages ready!


In [7]:
import os
import gc
import glob
import subprocess
from pathlib import Path
from tqdm import tqdm
import numpy as np
import h5py
import sqlite3
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from lightglue import ALIKED, LightGlue
from transformers import AutoImageProcessor, AutoModel
import pycolmap
from PIL import Image


class CONFIG:
    GLOBAL_TOPK = 200
    RATIO_THR = 1.2
    MATCH_THRESH = 10
    N_KEYPOINTS = 2048
    exhaustive_if_less = 20
    min_matches = 15
    max_num_keypoints = 8192
    image_size = 1024
    colmap_camera_model = 'SIMPLE_RADIAL'

In [8]:
# =========================================================
# COLMAP Database Utilities
# =========================================================
class COLMAPDatabase:
    @staticmethod
    def connect(database_path):
        return COLMAPDatabase(database_path)

    def __init__(self, database_path):
        self.connection = sqlite3.connect(database_path)
        self.cursor = self.connection.cursor()

    def create_tables(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS cameras (
                camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                model INTEGER NOT NULL,
                width INTEGER NOT NULL,
                height INTEGER NOT NULL,
                params BLOB,
                prior_focal_length INTEGER NOT NULL
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS images (
                image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                name TEXT NOT NULL UNIQUE,
                camera_id INTEGER NOT NULL,
                prior_qw REAL,
                prior_qx REAL,
                prior_qy REAL,
                prior_qz REAL,
                prior_tx REAL,
                prior_ty REAL,
                prior_tz REAL,
                CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < 2147483647),
                FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS keypoints (
                image_id INTEGER PRIMARY KEY NOT NULL,
                rows INTEGER NOT NULL,
                cols INTEGER NOT NULL,
                data BLOB,
                FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS matches (
                pair_id INTEGER PRIMARY KEY NOT NULL,
                rows INTEGER NOT NULL,
                cols INTEGER NOT NULL,
                data BLOB
            )
        """)
        self.cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)")

    def add_camera(self, model, width, height, params, prior_focal_length=1):
        params_blob = np.array(params, dtype=np.float64).tobytes()
        self.cursor.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (None, model, width, height, params_blob, prior_focal_length)
        )
        return self.cursor.lastrowid

    def add_image(self, name, camera_id, prior_q=None, prior_t=None):
        if prior_q is None:
            prior_q = [1.0, 0.0, 0.0, 0.0]
        if prior_t is None:
            prior_t = [0.0, 0.0, 0.0]

        self.cursor.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (None, name, camera_id, *prior_q, *prior_t)
        )
        return self.cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        if keypoints.dtype != np.float32:
            keypoints = keypoints.astype(np.float32)

        self.cursor.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id, keypoints.shape[0], keypoints.shape[1], keypoints.tobytes())
        )

    def add_matches(self, image_id1, image_id2, matches):
        pair_id = self.image_ids_to_pair_id(image_id1, image_id2)

        if matches.dtype != np.uint32:
            matches = matches.astype(np.uint32)

        self.cursor.execute(
            "INSERT OR REPLACE INTO matches VALUES (?, ?, ?, ?)",
            (pair_id, matches.shape[0], matches.shape[1], matches.tobytes())
        )

    @staticmethod
    def image_ids_to_pair_id(image_id1, image_id2):
        if image_id1 > image_id2:
            image_id1, image_id2 = image_id2, image_id1
        return image_id1 * 2147483648 + image_id2

    def commit(self):
        self.connection.commit()

    def close(self):
        self.connection.close()


# =========================================================
# H5 to Database Import
# =========================================================
CAMERA_MODEL_IDS = {
    'SIMPLE_PINHOLE': 0,
    'PINHOLE': 1,
    'SIMPLE_RADIAL': 2,
    'RADIAL': 3,
    'OPENCV': 4,
    'OPENCV_FISHEYE': 5,
}

def create_camera(db, image_path, camera_model):
    """Create camera entry"""
    img = Image.open(image_path)
    width, height = img.size

    # Simple radial model: f, cx, cy, k
    focal = max(width, height) * 1.2
    params = [focal, width/2, height/2, 0.0]

    model_id = CAMERA_MODEL_IDS.get(camera_model.upper(), 2)
    camera_id = db.add_camera(model_id, width, height, params)

    return camera_id

In [9]:
def import_into_colmap(image_dir, feature_dir, database_path):
    """COLMAP Database Import - Multiple Cameras Support"""
    print("\n=== Creating COLMAP Database ===")

    if os.path.exists(database_path):
        os.remove(database_path)

    # Create empty database structure
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create all tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cameras (
            camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            model INTEGER NOT NULL,
            width INTEGER NOT NULL,
            height INTEGER NOT NULL,
            params BLOB,
            prior_focal_length INTEGER NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS images (
            image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name TEXT NOT NULL UNIQUE,
            camera_id INTEGER NOT NULL,
            prior_qw REAL,
            prior_qx REAL,
            prior_qy REAL,
            prior_qz REAL,
            prior_tx REAL,
            prior_ty REAL,
            prior_tz REAL,
            FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS keypoints (
            image_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS matches (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS two_view_geometries (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            config INTEGER NOT NULL,
            F BLOB,
            E BLOB,
            H BLOB,
            qvec BLOB,
            tvec BLOB
        )
    """)

    # Load keypoints file
    kpts_file = os.path.join(feature_dir, 'keypoints.h5')
    matches_file = os.path.join(feature_dir, 'matches.h5')

    # Create cameras based on image sizes
    size_to_camera = {}  # (width, height) -> camera_id
    fname_to_id = {}
    image_id = 1

    with h5py.File(kpts_file, 'r') as f:
        print(f"Importing {len(f.keys())} images...")

        for filename in tqdm(f.keys(), desc="Adding images"):
            # Get image size
            image_path = os.path.join(image_dir, filename)
            try:
                img = Image.open(image_path)
                width, height = img.size
                img.close()
            except Exception as e:
                print(f"Warning: Cannot open {filename}: {e}")
                continue

            # Get or create camera for this size
            size_key = (width, height)
            if size_key not in size_to_camera:
                focal = max(width, height) * 1.2
                params = np.array([focal, width/2, height/2, 0.0], dtype=np.float64)

                cursor.execute(
                    "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
                    (None, 2, width, height, params.tobytes(), 1)  # 2 = SIMPLE_RADIAL
                )
                camera_id = cursor.lastrowid
                size_to_camera[size_key] = camera_id
            else:
                camera_id = size_to_camera[size_key]

            # Add image
            cursor.execute(
                "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (image_id, filename, camera_id, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
            )
            fname_to_id[filename] = image_id

            # Add keypoints
            kpts = f[filename][()].astype(np.float32)
            if len(kpts.shape) == 1:
                kpts = kpts.reshape(-1, 2)

            cursor.execute(
                "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
                (image_id, kpts.shape[0], 2, kpts.tobytes())
            )

            image_id += 1

    print(f"\nCreated {len(size_to_camera)} camera(s) for different image sizes:")
    for size, cam_id in sorted(size_to_camera.items()):
        print(f"  Camera {cam_id}: {size[0]}x{size[1]}")

    # Add matches
    total_matches = 0
    total_match_count = 0
    with h5py.File(matches_file, 'r') as f:
        print(f"\nProcessing matches...")
        for key1 in tqdm(f.keys(), desc="Adding matches"):
            if key1 not in fname_to_id:
                continue
            for key2 in f[key1].keys():
                if key2 not in fname_to_id:
                    continue

                id1, id2 = fname_to_id[key1], fname_to_id[key2]
                if id1 >= id2:
                    continue

                matches = f[key1][key2][()].astype(np.uint32)
                if matches.shape[0] == 0:
                    continue

                pair_id = id1 * 2147483648 + id2
                cursor.execute(
                    "INSERT INTO matches VALUES (?, ?, ?, ?)",
                    (pair_id, matches.shape[0], 2, matches.tobytes())
                )
                total_matches += 1
                total_match_count += matches.shape[0]

    conn.commit()
    conn.close()

    print(f"\n✓ Database created: {database_path}")
    print(f"  Cameras: {len(size_to_camera)}")
    print(f"  Images: {len(fname_to_id)}")
    print(f"  Match pairs: {total_matches}")
    print(f"  Total matches: {total_match_count}")

    return fname_to_id

In [10]:
def load_torch_image(fname, device=torch.device('cuda')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


def extract_dino_embeddings(fnames, device=torch.device('cuda')):
    print("\n=== Stage 1: Extracting DINO Global Features ===")

    processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
    model = AutoModel.from_pretrained('facebook/dinov2-base')
    model = model.eval().to(device)

    global_descs = []
    for img_path in tqdm(fnames, desc="DINO extraction"):
        timg = load_torch_image(img_path, device)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_feat = F.normalize(
                outputs.last_hidden_state[:,1:].max(dim=1)[0],
                dim=1, p=2
            )
        global_descs.append(dino_feat.detach().cpu())

    global_descs = torch.cat(global_descs, dim=0)
    print(f"Extracted global features: {global_descs.shape}")

    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

    return global_descs


def build_topk_pairs(global_feats, device):
    print("\n=== Building Top-K Pairs from Global Features ===")

    g = global_feats.to(device)
    sim = g @ g.T
    sim.fill_diagonal_(-1)

    N = sim.size(0)
    k = min(CONFIG.GLOBAL_TOPK, N - 1)
    k = max(k, 1)

    topk_indices = torch.topk(sim, k, dim=1).indices.cpu()

    pairs = set()
    for i, neighbors in enumerate(topk_indices):
        for j in neighbors:
            j = j.item()
            if i < j:
                pairs.add((i, j))

    pairs = sorted(list(pairs))
    print(f"Initial pairs from global features: {len(pairs)}")
    return pairs


def extract_aliked_features(fnames, device=torch.device('cuda')):
    print("\n=== Stage 2: Extracting ALIKED Local Features ===")

    dtype = torch.float32
    extractor = ALIKED(
        model_name="aliked-n16",
        max_num_keypoints=CONFIG.max_num_keypoints,
        detection_threshold=0.01,
        resize=CONFIG.image_size
    ).eval().to(device, dtype)

    keypoints_dict = {}
    descriptors_dict = {}

    for img_path in tqdm(fnames, desc="ALIKED extraction"):
        key = os.path.basename(img_path)
        image = load_torch_image(img_path, device=device).to(dtype)

        with torch.inference_mode():
            feats = extractor.extract(image)
            kpts = feats['keypoints'].reshape(-1, 2).detach().cpu()
            descs = feats['descriptors'].reshape(-1, 128).detach().cpu()
            descs = F.normalize(descs, dim=1).half()

        keypoints_dict[key] = kpts.numpy()
        descriptors_dict[key] = descs

    print(f"Extracted features for {len(keypoints_dict)} images")

    del extractor
    torch.cuda.empty_cache()
    gc.collect()

    return keypoints_dict, descriptors_dict


def verify_pairs_with_local_features(pairs, fnames, descriptors_dict, device):
    print("\n=== Verifying Pairs with Local Features ===")

    verified_pairs = []

    for i, j in tqdm(pairs, desc="Local verification"):
        key1 = os.path.basename(fnames[i])
        key2 = os.path.basename(fnames[j])

        desc1 = descriptors_dict[key1].to(device)
        desc2 = descriptors_dict[key2].to(device)

        if desc1.size(0) == 0 or desc2.size(0) == 0:
            continue

        with torch.inference_mode():
            sim = desc1 @ desc2.T
            nn1 = torch.argmax(sim, dim=1)
            nn2 = torch.argmax(sim, dim=0)
            mutual = torch.arange(len(nn1), device=device) == nn2[nn1]
            n_matches = mutual.sum().item()

        if n_matches >= CONFIG.MATCH_THRESH:
            verified_pairs.append((i, j))

    print(f"Verified pairs: {len(verified_pairs)}")
    return verified_pairs

In [11]:
def match_with_lightglue(verified_pairs, fnames, keypoints_dict, descriptors_dict,
                         output_dir, device=torch.device('cuda')):
    """Perform detailed matching using LightGlue - Fully Corrected Version"""
    print("\n=== Stage 3: Matching with LightGlue ===")

    os.makedirs(output_dir, exist_ok=True)

    lg_matcher = KF.LightGlueMatcher(
        "aliked", {
            "width_confidence": -1,
            "depth_confidence": -1,
            "mp": True if 'cuda' in str(device) else False
        }
    ).eval().to(device).half()

    print("Loaded LightGlue model")

    # Save keypoints
    kpts_h5_path = os.path.join(output_dir, 'keypoints.h5')
    with h5py.File(kpts_h5_path, 'w') as f:
        for img_path in fnames:
            key = os.path.basename(img_path)
            f.create_dataset(key, data=keypoints_dict[key])

    # Save matches
    matches_h5_path = os.path.join(output_dir, 'matches.h5')
    matched_pairs = 0
    skipped_pairs = 0
    total_matches = 0

    with h5py.File(matches_h5_path, 'w') as f_match:
        for i, j in tqdm(verified_pairs, desc="LightGlue matching"):
            key1 = os.path.basename(fnames[i])
            key2 = os.path.basename(fnames[j])

            kp1 = torch.from_numpy(keypoints_dict[key1]).to(device).half()
            kp2 = torch.from_numpy(keypoints_dict[key2]).to(device).half()
            desc1 = descriptors_dict[key1].to(device)
            desc2 = descriptors_dict[key2].to(device)

            if len(kp1) == 0 or len(kp2) == 0:
                skipped_pairs += 1
                continue

            with torch.inference_mode():
                try:
                    dists, idxs = lg_matcher(
                        desc1, desc2,
                        KF.laf_from_center_scale_ori(kp1[None]),
                        KF.laf_from_center_scale_ori(kp2[None])
                    )

                    # Check if matches were found
                    if idxs.numel() == 0:
                        skipped_pairs += 1
                        continue

                    # ★★★ Fix: Removed [0] ★★★
                    matches = idxs.cpu().numpy()  # (num_matches, 2)

                    # Check match count
                    num_matches = matches.shape[0]

                    if num_matches >= CONFIG.min_matches:
                        grp = f_match.require_group(key1)
                        grp.create_dataset(key2, data=matches)
                        matched_pairs += 1
                        total_matches += num_matches
                    else:
                        skipped_pairs += 1

                except Exception as e:
                    print(f"\nError matching {key1}-{key2}: {e}")
                    skipped_pairs += 1
                    continue

    del lg_matcher
    torch.cuda.empty_cache()
    gc.collect()

    print(f"\nMatching complete:")
    print(f"  Matched pairs: {matched_pairs}")
    print(f"  Skipped pairs: {skipped_pairs}")
    print(f"  Total matches: {total_matches}")
    print(f"  Average matches per pair: {total_matches/matched_pairs:.1f}" if matched_pairs > 0 else "")
    print(f"  Success rate: {matched_pairs/len(verified_pairs)*100:.1f}%")

    print(f"\nSaved keypoints to: {kpts_h5_path}")
    print(f"Saved matches to: {matches_h5_path}")


def run_colmap_sequential(database_path, image_dir, output_dir):
    """Run COLMAP reconstruction - GUI Error Prevention Version"""
    print("\n=== Stage 4: Running COLMAP Reconstruction ===")

    os.makedirs(output_dir, exist_ok=True)

    # Set environment variables to avoid GUI-related errors
    env = os.environ.copy()
    env['QT_QPA_PLATFORM'] = 'offscreen'

    # Skip sequential matcher (matches are already in the database)
    # Run the mapper directly
    print("Running mapper...")
    cmd_mapper = [
        'colmap', 'mapper',
        '--database_path', database_path,
        '--image_path', image_dir,
        '--output_path', output_dir,
        '--Mapper.ba_refine_focal_length', '0',
        '--Mapper.ba_refine_principal_point', '0',
        '--Mapper.ba_refine_extra_params', '0'
    ]

    try:
        result = subprocess.run(cmd_mapper, env=env, check=True,
                               capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("COLMAP mapper failed:")
        print(e.stdout)
        print(e.stderr)
        raise

    print(f"✓ COLMAP reconstruction saved to: {output_dir}")

In [12]:
def run_colmap_sequential(database_path, image_dir, output_dir):
    """Run COLMAP mapper with progress monitoring"""
    print("\n=== Stage 4: Running COLMAP Reconstruction ===")

    os.makedirs(output_dir, exist_ok=True)

    env = os.environ.copy()
    env['QT_QPA_PLATFORM'] = 'offscreen'

    from datetime import datetime, timezone
    import threading
    import time

    # Progress indicator thread
    def print_progress():
        start_time = time.time()
        while not stop_progress:
            elapsed = time.time() - start_time
            mins = int(elapsed // 60)
            secs = int(elapsed % 60)
            print(f"⏱️  Elapsed: {mins}m {secs}s | Still running...", flush=True)
            time.sleep(30)  # 30秒ごとに進捗表示

    stop_progress = False
    progress_thread = threading.Thread(target=print_progress, daemon=True)

    print(f"🚀 Starting mapper at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    print("📊 This typically takes 20-60 minutes for 80 images")
    print("💡 Progress updates every 30 seconds...\n")

    cmd_mapper = [
        'colmap', 'mapper',
        '--database_path', database_path,
        '--image_path', image_dir,
        '--output_path', output_dir,
        '--Mapper.ba_refine_focal_length', '0',
        '--Mapper.ba_refine_principal_point', '0',
        '--Mapper.ba_refine_extra_params', '0',
        # Speed optimizations
        '--Mapper.ba_local_max_num_iterations', '15',
        '--Mapper.ba_global_max_num_iterations', '30',
        '--Mapper.init_min_tri_angle', '4'
    ]

    # Start progress indicator
    progress_thread.start()

    try:
        # Run with real-time output
        import subprocess
        process = subprocess.Popen(
            cmd_mapper,
            env=env,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )

        # Print output line by line
        output_lines = []
        for line in process.stdout:
            print(line.rstrip())
            output_lines.append(line)

        process.wait(timeout=7200)  # 2時間タイムアウト

        stop_progress = True

        if process.returncode == 0:
            print(f"\n✅ COLMAP reconstruction saved to: {output_dir}")
            print(f"🕐 Completed at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
        else:
            print(f"\n❌ COLMAP mapper failed with return code {process.returncode}")
            raise subprocess.CalledProcessError(process.returncode, cmd_mapper)

    except subprocess.TimeoutExpired:
        stop_progress = True
        print("\n⏱️  Mapper timed out after 2 hours")
        print(f"📁 Check {output_dir} for any partial results")
        process.kill()
        raise

    except Exception as e:
        stop_progress = True
        print(f"\n❌ Error: {e}")
        raise

    finally:
        stop_progress = True
        time.sleep(1)  # プログレススレッドの終了を待つ


def main_pipeline(image_dir, output_base_dir):
    """Complete pipeline"""

    from datetime import datetime, timezone

    print(f"\n🚀 Pipeline started at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get images
    img_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
    fnames = []
    for ext in img_extensions:
        fnames.extend(glob.glob(os.path.join(image_dir, ext)))
    fnames = sorted(fnames)
    print(f"\n📸 Found {len(fnames)} images")

    if len(fnames) == 0:
        raise ValueError("No images found!")

    # Create directories
    feature_dir = os.path.join(output_base_dir, 'features')
    colmap_dir = os.path.join(output_base_dir, 'colmap')
    sparse_dir = os.path.join(colmap_dir, 'sparse')
    os.makedirs(feature_dir, exist_ok=True)
    os.makedirs(colmap_dir, exist_ok=True)

    # Stages 1-3: Feature extraction and matching
    print(f"\n⏰ Stage 1 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    global_feats = extract_dino_embeddings(fnames, device)

    print(f"\n⏰ Stage 2 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    initial_pairs = build_topk_pairs(global_feats, device)
    keypoints_dict, descriptors_dict = extract_aliked_features(fnames, device)

    print(f"\n⏰ Stage 3 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    verified_pairs = verify_pairs_with_local_features(
        initial_pairs, fnames, descriptors_dict, device
    )
    match_with_lightglue(
        verified_pairs, fnames, keypoints_dict, descriptors_dict,
        feature_dir, device
    )

    from datetime import datetime, timezone
    print(datetime.now(timezone.utc))

    print(f"\n⏰ Stage 4 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    # Stage 4: COLMAP Database + Reconstruction
    database_path = os.path.join(colmap_dir, 'database.db')
    import_into_colmap(image_dir, feature_dir, database_path)
    run_colmap_sequential(database_path, image_dir, sparse_dir)

    print("\n" + "="*60)
    print(f"✅ Pipeline Complete!")
    print(f"🕐 Finished at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")
    print("="*60)


# Execute
IMAGE_DIR = "/content/drive/MyDrive/your_folder/grand_place2"
OUTPUT_DIR = "/content/output"

main_pipeline(IMAGE_DIR, OUTPUT_DIR)


Found 50 images

=== Stage 1: Extracting DINO Global Features ===


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

...

Adding images: 100%|██████████| 50/50 [00:00<00:00, 280.04it/s]



Created 44 camera(s)

Processing matches...


Adding matches: 100%|██████████| 48/48 [00:00<00:00, 160.25it/s]



✓ Database created: /content/output/colmap/database.db
  Cameras: 44
  Images: 50
  Match pairs (in two_view_geometries): 960

=== Stage 4: Running COLMAP Reconstruction ===
Running mapper...


KeyboardInterrupt: 